## Model

In [9]:
import torch
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from modelscope import AutoTokenizer, AutoModelForCausalLM

model_id = 'Qwen/Qwen3-8B'
# model_id = 'Qwen/Qwen3-0.6B'
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
qwen_model = AutoModelForCausalLM.from_pretrained(model_id, trust_remote_code=True, torch_dtype=torch.float16).cuda()

D:\Anaconda\envs\eval\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2025-06-16 10:32:41,802 - modelscope - WARNING - Using branch: master as version is unstable, use with caution
2025-06-16 10:32:43,525 - modelscope - WARNING - Using branch: master as version is unstable, use with caution


Loading checkpoint shards: 100%|██████████████████████████████████████████████████| 5/5 [00:20<00:00,  4.18s/it]


In [10]:
# def qwen_llm(prompt):
#     inputs = tokenizer(prompt, return_tensors='pt').to('cuda')
#     outputs = model.generate(**inputs, max_new_tokens=32768)
#     return tokenizer.decode(outputs[0], skip_special_tokens=True)

def qwen_llm(prompt_str: str) -> str:
    print("Qwen...")
    if hasattr(prompt_str, 'to_string'):
        prompt_str = prompt_str.to_string()
    # ✅ 确保输入是字符串
    assert isinstance(prompt_str, str), f"Expected string, got {type(prompt_str)}"
    messages = [
        {"role": "user", "content": prompt_str}
    ]
    text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True,
            enable_thinking=True
        )
    inputs = tokenizer([text], return_tensors='pt').to(qwen_model.device)
    outputs = qwen_model.generate(**inputs, max_new_tokens=32768)  # ✅ 注意 max_new_tokens 拼写
    # return tokenizer.decode(outputs[0], skip_special_tokens=True).split('<think>\n\n</think>\n\n')[-1]
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

## Vector Store

In [1]:
from pymilvus import connections, Collection
connections.connect("default", host="localhost", port="19530")
col_name = "AllPolicies"
# col_name = "CountryPolicy"
# col_name = "ShanghaiPolicy"
# col_name = "CentrePolicy"
col = Collection(col_name)
col.load()

In [2]:
col

<Collection>:
-------------
<name>: AllPolicies
<description>: Policy Paragraph Embeddings
<schema>: {'auto_id': False, 'description': 'Policy Paragraph Embeddings', 'fields': [{'name': 'para_id', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 100}, 'is_primary': True, 'auto_id': False}, {'name': 'text', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 8192}}, {'name': 'dense_vector', 'description': '', 'type': <DataType.FLOAT_VECTOR: 101>, 'params': {'dim': 1024}}, {'name': 'sparse_vector', 'description': '', 'type': <DataType.SPARSE_FLOAT_VECTOR: 104>}], 'enable_dynamic_field': False}

## Embedding Model

In [23]:
from FlagEmbedding import BGEM3FlagModel

bgem3_model = BGEM3FlagModel('BAAI/bge-m3',
                      use_fp16=False,
                      pooling_method='cls',
                      devices=['cuda:0'])

def get_embeddings(text):
    embeddings = bgem3_model.encode(
        text,
        return_dense=True,
        return_sparse=True,
        return_colbert_vecs=False
    )
    return embeddings

Fetching 30 files: 100%|████████████████████████████████████████████████████████████████| 30/30 [00:00<?, ?it/s]


## Retriever

In [13]:
from pymilvus import AnnSearchRequest, WeightedRanker

def hybrid_search(
    col,
    query_dense_embedding,
    query_sparse_embedding,
    sparse_weight=1.0,
    dense_weight=1.0,
    limit=5,
):
    print("Searching...")
    dense_req = AnnSearchRequest(
        [query_dense_embedding], "dense_vector", {"metric_type": "L2", "params": {}}, limit=limit
    )
    sparse_req = AnnSearchRequest(
        [query_sparse_embedding], "sparse_vector", {"metric_type": "IP", "params": {}}, limit=limit
    )
    rerank = WeightedRanker(sparse_weight, dense_weight)
    res = col.hybrid_search(
        [sparse_req, dense_req],
        rerank=rerank,
        limit=limit,
        output_fields=["para_id", "text"]
    )
    return [
        {"para_id": hit.entity.get("para_id"), "text": hit.entity.get("text")}
        for hit in res[0]
    ]

def hybrid_search_pipeline(query):
    # 在这里调用你的 embedding 模型（比如 bge-m3）
    print("Embedding...")
    query_embeddings = get_embeddings([query])
    query_dense_embeddings = query_embeddings['dense_vecs'][0]
    query_sparse_embeddings = query_embeddings.get('lexical_weights')[0]

    # 调用原始的搜索方法
    return hybrid_search(
        col,
        query_dense_embeddings,
        query_sparse_embeddings,
        sparse_weight=1.0,
        dense_weight=1.0,
        limit=50
    )


## Reranker

In [14]:
from pymilvus.model.reranker import BGERerankFunction

bge_rf = BGERerankFunction(
    model_name="BAAI/bge-reranker-v2-m3",  # Specify the model name. Defaults to `BAAI/bge-reranker-v2-m3`.
    device="cuda:0" # Specify the device to use, e.g., 'cpu' or 'cuda:0'
)


In [15]:
def rerank(results):
    print("Reranking...")
    documents = [p['text'] for p in results]
    rerank_results = bge_rf(query=query, documents=documents, top_k=5)
    return rerank_results

## Chain

In [16]:
PROMPT_TEMPLATE = """
Human: You are an AI assistant, and provides answers to questions by using fact based and statistical information when possible.
Use the following pieces of information to provide a concise answer to the question enclosed in <question> tags.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
<context>
{context}
</context>

<question>
{question}
</question>

The response should be specific and use statistics or numbers when possible.

Assistant:"""

# Create a PromptTemplate instance with the defined template and input variables
prompt = PromptTemplate(
    template=PROMPT_TEMPLATE, input_variables=["context", "question"]
)


# Define a function to format the retrieved documents
def format_docs(docs):
    return "\n\n".join(doc.text for doc in docs)

In [17]:
from langchain_core.runnables import RunnableLambda

context_chain = RunnableLambda(hybrid_search_pipeline) | RunnableLambda(rerank) | RunnableLambda(format_docs)
llm = RunnableLambda(qwen_llm)

## Query

In [11]:
query = input("Enter your search query: ")
# query_embeddings = get_embeddings([query])

Enter your search query:  串通招标有什么后果？


In [12]:
rag_chain = (
    {"context": context_chain, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)


# res = rag_chain.invoke(query)
# res
for s in rag_chain.stream(query):
    print(s, end="", flush=True)

Embedding...


You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Searching...
Reranking...


You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Qwen...
user

Human: You are an AI assistant, and provides answers to questions by using fact based and statistical information when possible.
Use the following pieces of information to provide a concise answer to the question enclosed in <question> tags.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
<context>
第五十三条 第五十三条规定的情节严重行为，由有关行政监督部门取消其1年至2年内参加依法必须进行招标的项目的投标资格：(一)以行贿谋取中标；(二)3年内2次以上串通投标；(三)串通投标行为损害招标人、其他投标人或者国家、集体、公民的合法利益，造成直接经济损失30万元以上；(四)其他串通投标情节严重的行为。投标人自本条第二款规定的处罚执行期限届满之日起3年内又有该款所列违法行为之一的，或者串通投标、以行贿谋取中标情节特别严重的，由工商行政管理机关吊销营业执照。法律、行政法规对串通投标报价行为的处罚另有规定的，从其规定。

第五十三条 第五十三条投标人相互串通投标或者与招标人串通投标的，投标人以向招标人或者评标委员会成员行贿的手段谋取中标的，中标无效，处中标项目金额千分之五以上千分之十以下的罚款，对单位直接负责的主管人员和其他直接责任人员处单位罚款数额百分之五以上百分之十以下的罚款；有违法所得的，并处没收违法所得；情节严重的，取消其一年至二年内参加依法必须进行招标的项目的投标资格并予以公告，直至由工商行政管理机关吊销营业执照；构成犯罪的，依法追究刑事责任。给他人造成损失的，依法承担赔偿责任。

第七十四条 第七十四条投标人相互串通投标或者与招标人串通投标的，投标人以向招标人或者评标委员会成员行贿的手段谋取中标的，中标无效，由有关行政监督部门处中标项目金额千分之五以上千分之十以下的罚款，对单位直接负责的主管人员和其他直接责任人员处单位罚款数额百分之五以上百分之十以下的罚款

In [13]:
model_id

'Qwen/Qwen3-8B'

In [ ]:
def RAG()

## Others

In [7]:
import jieba

def doc_text_formatting(query, docs):
    query_words = list(set(jieba.lcut(query)))  # 中文分词
    formatted_texts = []

    for doc in docs:
        try:
            highlighted = doc.get('text')
        except:
            highlighted = doc.text
        for word in query_words:
            if not word.strip():
                continue
            highlighted = re.sub(
                re.escape(word),
                f"<span style='color:red'>{word}</span>",
                highlighted
            )
        formatted_texts.append(highlighted)
    return formatted_texts


In [13]:
query_dense_embeddings = query_embeddings['dense_vecs']
query_sparse_embeddings = query_embeddings.get('lexical_weights')

In [14]:
hybrid_results = hybrid_search(
        col,
        query_dense_embeddings[0],
        query_sparse_embeddings[0],
        sparse_weight=1.0,
        dense_weight=1.0,
        limit=50
    )

Searching...


In [15]:
import re
from IPython.display import Markdown, display

display(Markdown("### 🔄 **Hybrid Search Results:**"))
formatted_results = doc_text_formatting(query, hybrid_results)
for i, result in enumerate(formatted_results):
    display(Markdown(f"para_id: {hybrid_results[i].get('para_id')}"))
    display(Markdown(result))

### 🔄 **Hybrid Search Results:**

para_id: 8063182_T25

第二十一条 第二十一条<span style='color:red'>国有</span><span style='color:red'><span style='color:red'>不</span>可</span><span style='color:red'>移动</span><span style='color:red'>文物</span>由使用人负责<span style='color:red'>修缮</span><span style='color:red'>、</span><span style='color:red'>保养</span>；非<span style='color:red'>国有</span><span style='color:red'><span style='color:red'>不</span>可</span><span style='color:red'>移动</span><span style='color:red'>文物</span>由<span style='color:red'>所有人</span>负责<span style='color:red'>修缮</span><span style='color:red'>、</span><span style='color:red'>保养</span>。非<span style='color:red'>国有</span><span style='color:red'><span style='color:red'>不</span>可</span><span style='color:red'>移动</span><span style='color:red'>文物</span>有损毁危险<span style='color:red'>，</span><span style='color:red'>所有人</span><span style='color:red'>不</span><span style='color:red'>具备</span><span style='color:red'>修缮</span><span style='color:red'>能力</span><span style='color:red'>的</span><span style='color:red'>，</span>当地人民<span style='color:red'>政府</span><span style='color:red'>应</span>当给予帮助；<span style='color:red'>所有人</span><span style='color:red'>具备</span><span style='color:red'>修缮</span><span style='color:red'>能力</span>而拒<span style='color:red'>不</span>依法履行<span style='color:red'>修缮</span><span style='color:red'>义务</span><span style='color:red'>的</span><span style='color:red'>，</span><span style='color:red'>县级</span><span style='color:red'>以上</span>人民<span style='color:red'>政府</span>可以给予抢救<span style='color:red'>修缮</span><span style='color:red'>，</span>所需费用由<span style='color:red'>所有人</span>负担。<span style='color:red'>对</span><span style='color:red'>文物</span>保护单位<span style='color:red'>进行</span><span style='color:red'>修缮</span><span style='color:red'>，</span><span style='color:red'>应</span>当根据<span style='color:red'>文物</span>保护单位<span style='color:red'>的</span>级别报相<span style='color:red'>应</span><span style='color:red'>的</span><span style='color:red'>文物</span>行政部门批准；<span style='color:red'>对</span>未核定为<span style='color:red'>文物</span>保护单位<span style='color:red'>的</span><span style='color:red'><span style='color:red'>不</span>可</span><span style='color:red'>移动</span><span style='color:red'>文物</span><span style='color:red'>进行</span><span style='color:red'>修缮</span><span style='color:red'>，</span><span style='color:red'>应</span>当报登记<span style='color:red'>的</span><span style='color:red'>县级</span>人民<span style='color:red'>政府</span><span style='color:red'>文物</span>行政部门批准。<span style='color:red'>文物</span>保护单位<span style='color:red'>的</span><span style='color:red'>修缮</span><span style='color:red'>、</span><span style='color:red'>迁移</span><span style='color:red'>、</span><span style='color:red'>重建</span><span style='color:red'>，</span>由取得<span style='color:red'>文物</span>保护工程<span style='color:red'>资质</span>证书<span style='color:red'>的</span>单位承担。<span style='color:red'>对</span><span style='color:red'><span style='color:red'>不</span>可</span><span style='color:red'>移动</span><span style='color:red'>文物</span><span style='color:red'>进行</span><span style='color:red'>修缮</span><span style='color:red'>、</span><span style='color:red'>保养</span><span style='color:red'>、</span><span style='color:red'>迁移</span><span style='color:red'>，</span><span style='color:red'>必须</span><span style='color:red'>遵守</span><span style='color:red'>不</span>改变<span style='color:red'>文物</span>原状<span style='color:red'>的</span><span style='color:red'>原则</span>。

para_id: 8063182_T89

第七十五条 第七十五条有下列行为之一<span style='color:red'>的</span><span style='color:red'>，</span>由<span style='color:red'>县级</span><span style='color:red'>以上</span>人民<span style='color:red'>政府</span><span style='color:red'>文物</span>主管部门责令改正：（一）改变<span style='color:red'>国有</span>未核定为<span style='color:red'>文物</span>保护单位<span style='color:red'>的</span><span style='color:red'><span style='color:red'>不</span>可</span><span style='color:red'>移动</span><span style='color:red'>文物</span><span style='color:red'>的</span>用途<span style='color:red'>，</span>未依照本法规定报告<span style='color:red'>的</span>；（二）转让<span style='color:red'>、</span>抵押非<span style='color:red'>国有</span><span style='color:red'><span style='color:red'>不</span>可</span><span style='color:red'>移动</span><span style='color:red'>文物</span><span style='color:red'>或</span>者改变其用途<span style='color:red'>，</span>未依照本法规定备案<span style='color:red'>的</span>；（三）<span style='color:red'>国有</span><span style='color:red'><span style='color:red'>不</span>可</span><span style='color:red'>移动</span><span style='color:red'>文物</span><span style='color:red'>的</span>使用人拒<span style='color:red'>不</span>依法履行<span style='color:red'>修缮</span><span style='color:red'>义务</span><span style='color:red'>的</span>；（四）考古发掘单位未经批准擅自<span style='color:red'>进行</span>考古发掘<span style='color:red'>，</span><span style='color:red'>或</span>者<span style='color:red'>不</span>如实报告考古发掘结果<span style='color:red'>的</span>；（五）<span style='color:red'>文物</span>收藏单位未按照国家有关规定建立馆藏<span style='color:red'>文物</span>档案<span style='color:red'>、</span>管理制度<span style='color:red'>，</span><span style='color:red'>或</span>者未将馆藏<span style='color:red'>文物</span>档案<span style='color:red'>、</span>管理制度备案<span style='color:red'>的</span>；（六）违反本法

...

第十八条 第十八条根据保护<span style='color:red'>文物</span><span style='color:red'>的</span>实际<span style='color:red'>需要</span><span style='color:red'>，</span>经省<span style='color:red'>、</span>自治区<span style='color:red'>、</span>直辖市人民<span style='color:red'>政府</span>批准<span style='color:red'>，</span>可以<span style='color:red'>在</span><span style='color:red'>文物</span>保护单位<span style='color:red'>的</span>周围划出一定<span style='color:red'>的</span>建设控制地带<span style='color:red'>，</span>并予以公布。<span style='color:red'>在</span><span style='color:red'>文物</span>保护单位<span style='color:red'>的</span>建设控制地带内<span style='color:red'>进行</span>建设工程<span style='color:red'>，</span><span style='color:red'>不</span>得破坏<span style='color:red'>文物</span>保护单位<span style='color:red'>的</span>历史风貌；工程设计方案<span style='color:red'>应</span>当根据<span style='color:red'>文物</span>保护单位<span style='color:red'>的</span>级别<span style='color:red'>，</span>经相<span style='color:red'>应</span><span style='color:red'>的</span><span style='color:red'>文物</span>行政部门同意后<span style='color:red'>，</span>报城乡建设规划部门批准。

para_id: 8063182_T34

第三十条 第三十条<span style='color:red'>需要</span>配合建设工程<span style='color:red'>进行</span><span style='color:red'>的</span>考古发掘工作<span style='color:red'>，</span><span style='color:red'>应</span>当由省<span style='color:red'>、</span>自治区<span style='color:red'>、</span>直辖市<span style='color:red'>文物</span>行政部门<span style='color:red'>在</span>勘探工作<span style='color:red'>的</span>基础上提出发掘计划<span style='color:red'>，</span>报国务院<span style='color:red'>文物</span>行政部门批准。国务院<span style='color:red'>文物</span>行政部门<span style='color:red'>在</span>批准前<span style='color:red'>，</span><span style='color:red'>应</span>当征求社会科学研究机构及其他科研机构<span style='color:red'>和</span>有关专家<span style='color:red'>的</span>意见。确因建设工期紧迫<span style='color:red'>或</span>者有自然破坏危险<span style='color:red'>，</span><span style='color:red'>对</span>古文化遗址<span style='color:red'>、</span>古墓葬急需<span style='color:red'>进行</span>抢救发掘<span style='color:red'>的</span><span style='color:red'>，</span>由省<span style='color:red'>、</span>自治区<span style='color:red'>、</span>直辖市人民<span style='color:red'>政府</span><span style='color:red'>文物</span>行政部门组织发掘<span style='color:red'>，</span>并同<span style='color:red'>时</span>补办<span style='color:red'>审批</span>手续。

para_id: 8063182_T10

第六条 第六条属于集体所有<span style='color:red'>和</span>私人所有<span style='color:red'>的</span>纪念建筑物<span style='color:red'>、</span>古建筑<span style='color:red'>和</span>祖传<span style='color:red'>文物</span>以及依法取得<span style='color:red'>的</span>其他<span style='color:red'>文物</span><span style='color:red'>，</span>其所有权受法律保护。<span style='color:red'>文物</span><span style='color:red'>的</span>所有者<span style='color:red'>必须</span><span style='color:red'>遵守</span>国家有关<span style='color:red'>文物</span>保护<span style='color:red'>的</span>法律<span style='color:red'>、</span>法规<span style='color:red'>的</span>规定。

In [18]:
hybrid_rerank_results = rerank(hybrid_results)

Reranking...


You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [19]:
hybrid_rerank_results

[RerankResult(text='第二十一条 第二十一条国有不可移动文物由使用人负责修缮、保养；非国有不可移动文物由所有人负责修缮、保养。非国有不可移动文物有损毁危险，所有人不具备修缮能力的，当地人民政府应当给予帮助；所有人具备修缮能力而拒不依法履行修缮义务的，县级以上人民政府可以给予抢救修缮，所需费用由所有人负担。对文物保护单位进行修缮，应当根据文物保护单位的级别报相应的文物行政部门批准；对未核定为文物保护单位的不可移动文物进行修缮，应当报登记的县级人民政府文物行政部门批准。文物保护单位的修缮、迁移、重建，由取得文物保护工程资质证书的单位承担。对不可移动文物进行修缮、保养、迁移，必须遵守不改变文物原状的原则。', score=0.9993952897517442, index=0),
 RerankResult(text='第二十六条 第二十六条使用不可移动文物，必须遵守不改变文物原状的原则，负责保护建筑物及其附属文物的安全，不得损毁、改建、添建或者拆除不可移动文物。对危害文物保护单位安全、破坏文物保护单位历史风貌的建筑物、构筑物，当地人民政府应当及时调查处理，必要时，对该建筑物、构筑物予以拆迁。第三章\u3000考古发掘', score=0.6842640244786128, index=2),
 RerankResult(text='第二十二条 第二十二条不可移动文物已经全部毁坏的，应当实施遗址保护，不得在原址重建。但是，因特殊情况需要在原址重建的，由省、自治区、直辖市人民政府文物行政部门报省、自治区、直辖市人民政府批准；全国重点文物保护单位需要在原址重建的，由省、自治区、直辖市人民政府报国务院批准。', score=0.5710491189132763, index=6),
 RerankResult(text='第五条 第五条中华人民共和国境内地下、内水和领海中遗存的一切文物，属于国家所有。古文化遗址、古墓葬、石窟寺属于国家所有。国家指定保护的纪念建筑物、古建筑、石刻、壁画、近代现代代表性建筑等不可移动文物，除国家另有规定的以外，属于国家所有。国有不可移动文物的所有权不因其所依附的土地所有权或者使用权的改变而改变。下列可移动文物，属于国家所有：（一）中国境内出土的文物，国家另有规定的除外；（二）国有文物收藏单位以及其他国家机关、部队和国有企业、事业组

In [20]:
display(Markdown("### 🔄 **Reranked Hybrid Search Results:**"))
formatted_results = doc_text_formatting(query, hybrid_rerank_results)
for i, result in enumerate(formatted_results):
    display(Markdown(f"para_id: {hybrid_results[i].get('para_id')}"))
    display(Markdown(result))

### 🔄 **Reranked Hybrid Search Results:**

para_id: 8063182_T25

第二十一条 第二十一条<span style='color:red'>国有</span><span style='color:red'><span style='color:red'>不</span>可</span><span style='color:red'>移动</span><span style='color:red'>文物</span>由使用人负责<span style='color:red'>修缮</span><span style='color:red'>、</span><span style='color:red'>保养</span>；非<span style='color:red'>国有</span><span style='color:red'><span style='color:red'>不</span>可</span><span style='color:red'>移动</span><span style='color:red'>文物</span>由<span style='color:red'>所有人</span>负责<span style='color:red'>修缮</span><span style='color:red'>、</span><span style='color:red'>保养</span>。非<span style='color:red'>国有</span><span style='color:red'><span style='color:red'>不</span>可</span><span style='color:red'>移动</span><span style='color:red'>文物</span>有损毁危险<span style='color:red'>，</span><span style='color:red'>所有人</span><span style='color:red'>不</span><span style='color:red'>具备</span><span style='color:red'>修缮</span><span style='color:red'>能力</span><span style='color:red'>的</span><span style='color:red'>，</span>当地人民<span style='color:red'>政府</span><span style='color:red'>应</span>当给予帮助；<span style='color:red'>所有人</span><span style='color:red'>具备</span><span style='color:red'>修缮</span><span style='color:red'>能力</span>而拒<span style='color:red'>不</span>依法履行<span style='color:red'>修缮</span><span style='color:red'>义务</span><span style='color:red'>的</span><span style='color:red'>，</span><span style='color:red'>县级</span><span style='color:red'>以上</span>人民<span style='color:red'>政府</span>可以给予抢救<span style='color:red'>修缮</span><span style='color:red'>，</span>所需费用由<span style='color:red'>所有人</span>负担。<span style='color:red'>对</span><span style='color:red'>文物</span>保护单位<span style='color:red'>进行</span><span style='color:red'>修缮</span><span style='color:red'>，</span><span style='color:red'>应</span>当根据<span style='color:red'>文物</span>保护单位<span style='color:red'>的</span>级别报相<span style='color:red'>应</span><span style='color:red'>的</span><span style='color:red'>文物</span>行政部门批准；<span style='color:red'>对</span>未核定为<span style='color:red'>文物</span>保护单位<span style='color:red'>的</span><span style='color:red'><span style='color:red'>不</span>可</span><span style='color:red'>移动</span><span style='color:red'>文物</span><span style='color:red'>进行</span><span style='color:red'>修缮</span><span style='color:red'>，</span><span style='color:red'>应</span>当报登记<span style='color:red'>的</span><span style='color:red'>县级</span>人民<span style='color:red'>政府</span><span style='color:red'>文物</span>行政部门批准。<span style='color:red'>文物</span>保护单位<span style='color:red'>的</span><span style='color:red'>修缮</span><span style='color:red'>、</span><span style='color:red'>迁移</span><span style='color:red'>、</span><span style='color:red'>重建</span><span style='color:red'>，</span>由取得<span style='color:red'>文物</span>保护工程<span style='color:red'>资质</span>证书<span style='color:red'>的</span>单位承担。<span style='color:red'>对</span><span style='color:red'><span style='color:red'>不</span>可</span><span style='color:red'>移动</span><span style='color:red'>文物</span><span style='color:red'>进行</span><span style='color:red'>修缮</span><span style='color:red'>、</span><span style='color:red'>保养</span><span style='color:red'>、</span><span style='color:red'>迁移</span><span style='color:red'>，</span><span style='color:red'>必须</span><span style='color:red'>遵守</span><span style='color:red'>不</span>改变<span style='color:red'>文物</span>原状<span style='color:red'>的</span><span style='color:red'>原则</span>。

para_id: 8063182_T89

第二十六条 第二十六条使用<span style='color:red'><span style='color:red'>不</span>可</span><span style='color:red'>移动</span><span style='color:red'>文物</span><span style='color:red'>，</span><span style='color:red'>必须</span><span style='color:red'>遵守</span><span style='color:red'>不</span>改变<span style='color:red'>文物</span>原状<span style='color:red'>的</span><span style='color:red'>原则</span><span style='color:red'>，</span>负责保护建筑物及其附属<span style='color:red'>文物</span><span style='color:red'>的</span>安全<span style='color:red'>，</span><span style='color:red'>不</span>得损毁<span style='color:red'>、</span>改建<span style='color:red'>、</span>添建<span style='color:red'>或</span>者拆除<span style='color:red'><span style='color:red'>不</span>可</span><span style='color:red'>移动</span><span style='color:red'>文物</span>。<span style='color:red'>对</span>危害<span style='color:red'>文物</span>保护单位安全<span style='color:red'>、</span>破坏<span style='color:red'>文物</span>保护单位历史风貌<span style='color:red'>的</span>建筑物<span style='color:red'>、</span>构筑物<span style='color:red'>，</span>当地人民<span style='color:red'>政府</span><span style='color:red'>应</span>当及<span style='color:red'>时</span>调查处理<span style='color:red'>，</span>必要<span style='color:red'>时</span><span style='color:red'>，</span><span style='color:red'>对</span>该建筑物<span style='color:red'>、</span>构筑物予以拆迁。第三章　考古发掘

...

第二十二条 第二十二条<span style='color:red'><span style='color:red'>不</span>可</span><span style='color:red'>移动</span><span style='color:red'>文物</span>已经全部毁坏<span style='color:red'>的</span><span style='color:red'>，</span><span style='color:red'>应</span>当实施遗址保护<span style='color:red'>，</span><span style='color:red'>不</span>得<span style='color:red'>在</span>原址<span style='color:red'>重建</span>。但是<span style='color:red'>，</span>因特殊情况<span style='color:red'>需要</span><span style='color:red'>在</span>原址<span style='color:red'>重建</span><span style='color:red'>的</span><span style='color:red'>，</span>由省<span style='color:red'>、</span>自治区<span style='color:red'>、</span>直辖市人民<span style='color:red'>政府</span><span style='color:red'>文物</span>行政部门报省<span style='color:red'>、</span>自治区<span style='color:red'>、</span>直辖市人民<span style='color:red'>政府</span>批准；全国重点<span style='color:red'>文物</span>保护单位<span style='color:red'>需要</span><span style='color:red'>在</span>原址<span style='color:red'>重建</span><span style='color:red'>的</span><span style='color:red'>，</span>由省<span style='color:red'>、</span>自治区<span style='color:red'>、</span>直辖市人民<span style='color:red'>政府</span>报国务院批准。

para_id: 8063182_T24

第五条 第五条中华人民共<span style='color:red'>和</span>国境内地下<span style='color:red'>、</span>内水<span style='color:red'>和</span>领海中遗存<span style='color:red'>的</span>一切<span style='color:red'>文物</span><span style='color:red'>，</span>属于国家所有。古文化遗址<span style='color:red'>、</span>古墓葬<span style='color:red'>、</span>石窟寺属于国家所有。国家指定保护<span style='color:red'>的</span>纪念建筑物<span style='color:red'>、</span>古建筑<span style='color:red'>、</span>石刻<span style='color:red'>、</span>壁画<span style='color:red'>、</span>近代现代代表性建筑等<span style='color:red'><span style='color:red'>不</span>可</span><span style='color:red'>移动</span><span style='color:red'>文物</span><span style='color:red'>，</span>除国家另有规定<span style='color:red'>的</span>以外<span style='color:red'>，</span>属于国家所有。<span style='color:red'>国有</span><span style='color:red'><span style='color:red'>不</span>可</span><span style='color:red'>移动</span><span style='color:red'>文物</span><span style='color:red'>的</span>所有权<span style='color:red'>不</span>因其所依附<span style='color:red'>的</span>土地所有权<span style='color:red'>或</span>者使用权<span style='color:red'>的</span>改变而改变。下列可<span style='color:red'>移动</span><span style='color:red'>文物</span><span style='color:red'>，</span>属于国家所有：（一）中国境内出土<span style='color:red'>的</span><span style='color:red'>文物</span><span style='color:red'>，</span>国家另有规定<span style='color:red'>的</span>除外；（二）<span style='color:red'>国有</span><span style='color:red'>文物</span>收藏单位以及其他国家机关<span style='color:red'>、</span>部队<span style='color:red'>和</span><span style='color:red'>国有</span>企业<span style='color:red'>、</span>事业组织等收藏<span style='color:red'>、</span>保管<span style='color:red'>的</span><span style='color:red'>文物</span>；（三）国家征集<span style='color:red'>、</span>购买<span style='color:red'>的</span><span style='color:red'>文物</span>；（四）公民<span style='color:red'>、</span>法人<span style='color:red'>和</span>其他组织捐赠给国家<span style='color:red'>的</span><span style='color:red'>文物</span>；（五）法律规定属于国家所有<span style='color:red'>的</span>其他<span style='color:red'>文物</span>。属于国家所有<span style='color:red'>的</span>可<span style='color:red'>移动</span><span style='color:red'>文物</span><span style='color:red'>的</span>所有权<span style='color:red'>不</span>因其保管<span style='color:red'>、</span>收藏单位<span style='color:red'>的</span>终止<span style='color:red'>或</span>者变更而改变。<span style='color:red'>国有</span><span style='color:red'>文物</span>所有权受法律保护<span style='color:red'>，</span><span style='color:red'>不</span>容侵犯。

para_id: 8063182_T75

第三十一条 第三十一条<span style='color:red'>的</span>决定》第一次修正　2002年10月28日第九届全国人民代表大会常务委员会第三十次会议修订　根据2007年12月29日第十届全国人民代表大会常务委员会第三十一次会议《关于修改〈中华人民共<span style='color:red'>和</span>国<span style='color:red'>文物</span>保护法〉<span style='color:red'>的</span>决定》第二次修正　根据2013年6月29日第十二届全国人民代表大会常务委员会第三次会议《关于修改〈中华人民共<span style='color:red'>和</span>国<span style='color:red'>文物</span>保护法〉等十二部法律<span style='color:red'>的</span>决定》第三次修正　根据2015年4月24日第十二届全国人民代表大会常务委员会第十四次会议《关于修改〈中华人民共<span style='color:red'>和</span>国<span style='color:red'>文物</span>保护法〉<span style='color:red'>的</span>决定》第四次修正　根据2017年11月4日第十二届全国人民代表大会常务委员会第三十次会议《关于修改〈中华人民共<span style='color:red'>和</span>国会计法〉等十一部法律<span style='color:red'>的</span>决定》第五次修正）中华人民共<span style='color:red'>和</span>国<span style='color:red'>文物</span>保护法(2017修正)目录第一章　总 则第二章　<span style='color:red'><span style='color:red'>不</span>可</span><span style='color:red'>移动</span><span style='color:red'>文物</span>第三章　考古发掘第四章　馆藏<span style='color:red'>文物</span>第五章　民间收藏<span style='color:red'>文物</span>第六章　<span style='color:red'>文物</span>出境进境第七章　法律<span style='color:red'>责任</span>第八章　附 则第一章　总 则

In [24]:
prompt

PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="\nHuman: You are an AI assistant, and provides answers to questions by using fact based and statistical information when possible.\nUse the following pieces of information to provide a concise answer to the question enclosed in <question> tags.\nIf you don't know the answer, just say that you don't know, don't try to make up an answer.\n<context>\n{context}\n</context>\n\n<question>\n{question}\n</question>\n\nThe response should be specific and use statistics or numbers when possible.\n\nAssistant:")

In [27]:
rag_sub_chain = (
    {"context": RunnableLambda(hybrid_results) | RunnableLambda(format_docs), "question": RunnablePassthrough()}
    | prompt
    | StrOutputParser()
)

TypeError: Expected a callable type for `func`.Instead got an unsupported type: <class 'list'>